
Copyright(c) 2025 NVIDIA Corporation. All rights reserved.

NVIDIA Corporation and its licensors retain all intellectual property
and proprietary rights in and to this software, related documentation
and any modifications thereto.Any use, reproduction, disclosure or
distribution of this software and related documentation without an express
license agreement from NVIDIA Corporation is strictly prohibited.

# Enable speculative speech processing for the Digitial Human Pipeline. 

This notebook provides step-by-step instructions for enabling the speculative speech processing for the Digital Human Pipeline. Speculative speech processing enables real-time, natural conversational AI by processing interim transcripts and generating early responses while maintaining conversation coherence. It uses NVIDIA’s specialized frame processors and context aggregators.


### Ensure that the Pre-requisites are met

Please run through the steps noted in the setup prequisites notebook to ensure that you have the necessary setup in place.

Check the status of the application again by running the following command. 

Note: You will need to select the python environment to run the notebook for the first time. 

In [ ]:
!kubectl get po -n app

### Download the ACE Controller Configuration
* Ensure that the downloaded ACE Controller configuration is in sync. You can click on the download button again to download the latest version of the configuration. 

![ACE Configurator](pictures/ace_configurator_sync.png)

###  Update the bot pipeline
1. Navigate to the following path in your file system:
   ```
   ace-controller-ace-controller-deployment -> assets -> app-storage-volume -> src -> bot.py
   ```
2. Open the `bot.py` file.

![Avatar Controller Config](pictures/ace_controller_bot.png)

3. The speculative speech processing is disabled by default. You can enable it by uncommenting/commenting the lines as guided in the code. Snippets shown below are from the `bot.py` file. 

In [ ]:
from pipecat_nvidia.processors.nvidia_context_aggregator import (
    create_nvidia_context_aggregator,
    # NvidiaTTSResponseCacher, # Uncomment to enable speculative speech processing
)


# Comment out the below line when enabling Speculative Speech Processing
context_aggregator = llm.create_context_aggregator(context)

# Uncomment the below line to enable speculative speech processing
# nvidia_context_aggregator = create_nvidia_context_aggregator(context, send_interims=True)
# Uncomment the below line to enable speculative speech processing
# nvidia_tts_response_cacher = NvidiaTTSResponseCacher()


pipeline = Pipeline(
            [
                transport.input(),
                user_presence_processor,
                stt, # Speech-To-Text
                stt_transcript_synchronization,
                # Comment out the below line when enabling Speculative Speech Processing
                context_aggregator.user(),
                # Uncomment the below line to enable speculative speech processing
                # nvidia_context_aggregator.user(),
                llm,
                proactivity_processor,
                tts,
                # Uncomment the below line to enable speculative speech processing
                # nvidia_tts_response_cacher,  # For caching TTS response when Speculative Speech Processing is enabled
                tts_transcript_synchronization,
                custom_view_processor,
                facial_gestures,
                a2f,
                posture_provider_processor,
                animgraph,
                # Comment out the below line when enabling Speculative Speech Processing
                context_aggregator.assistant(),
                # Uncomment the below line to enable speculative speech processing
                # nvidia_context_aggregator.assistant(),
                transport.output(),  # Websocket output to client
            ]
        )



### Upload the Changes
Save your local changes in VSCode. 
In the ACE Configurator view in VSCode, click the **Upload** button to upload the modified `bot.py` file.

![Avatar Controller Upload Config](pictures/upload_config.png)

Ensure that the changes are finished updating. This might take a few seconds.

Once the changes are successfully updated. the ACE Configurator view will show a green `Synced` symbol again for the app-storage-volume. 

![Avatar Controller Upload Config](pictures/ace-configurator-synced.png)


### Refresh the Browser UI

1. Open the browser UI where the digital human is hosted. (your current URL:30111)
2. Refresh the page.(Note: you might need to refresh it a couple of times)
3. Start a conversation with the digital human.

###  Test the speculative speech processing

Observe that the digital human responds latency is reduced for user queries.


### Conclusion
You have successfully reconfigured the ACE digital human and tested the changes. 🎉